In [2]:
import numpy as np
import scipy as sy
import matplotlib.pyplot as plt
import sys
import os
import ast
import json
import random
from collections import defaultdict
from collections import Counter
from sklearn.model_selection import train_test_split
from ipynb.fs.full.Random_Sample_Mapper import *

In [3]:
notebook_path = os.path.abspath("BPR_OPT_Binary_Model.ipynb")
users_items_file_path = os.path.join(os.path.dirname(notebook_path), "data/australian_users_items.json")
items_file_path = os.path.join(os.path.dirname(notebook_path), "data/items_meta_data.json")
users_meta_data_file_path = os.path.join(os.path.dirname(notebook_path), "data/users_meta_data.json")

In [4]:
users_items = []
with open(users_items_file_path, 'r') as data:
    for line in data:
        users_items.append(ast.literal_eval(line))

In [5]:
with open(items_file_path, 'r') as data:
    games_dict = json.load(data)

In [6]:
with open(users_meta_data_file_path, 'r') as file:
    users_meta_data = json.load(file)

In [7]:
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
playtimesPerItem = defaultdict(dict)
playtimesPerUser = defaultdict(dict)
itemNames = defaultdict(str)

In [8]:
train_usersPerItem = defaultdict(set)
train_itemsPerUser = defaultdict(set)
test_usersPerItem = defaultdict(set)
test_itemsPerUser = defaultdict(set)

In [9]:
for game in games_dict:
    if 'owners' in games_dict[game]:
        usersPerItem[game] = set(games_dict[game]['owners'].keys())

In [10]:
for user in users_items:
    u_id = user['user_id']
    items = [item['item_id'] for item in user['items']]
    itemsPerUser[u_id] = items
    
    items_train, items_test = train_test_split(items, test_size = 0.995)
    train_itemsPerUser[u_id] = items_train
    test_itemsPerUser[u_id] = items_test

    playtimesPerUser[user['user_id']] = dict((item['item_id'], item['playtime_forever']) for item in user['items'])
    for item in user['items']:
        itemNames[item['item_id']] = item['item_name']
        playtimesPerItem[item['item_id']][user['user_id']] = item['playtime_forever']

In [11]:
for user in train_itemsPerUser:
    for item in train_itemsPerUser[user]:
        train_usersPerItem[item].add(user)
        
    for item in test_itemsPerUser[user]:
        test_usersPerItem[item].add(user)

In [29]:
train_nUsers = len(train_itemsPerUser)
train_nItems = len(train_usersPerItem)
train_users = list(train_itemsPerUser.keys())
train_items = list(train_usersPerItem.keys())

In [30]:
user_item_counts = dict((k, len(v)) for k, v in train_itemsPerUser.items())

In [31]:
datafile = 'data/sample_in.tsv'
mapout1 = 'data/sample_map1.tsv'
mapout2 = 'data/sample_map2.tsv'
outfile = 'data/sample_out.tsv'

f = open(datafile,'w')
for user, items in train_itemsPerUser.items():
    for item in items:
        print(default_formatter(user,item), file=f)
f.close()

In [32]:
# run two stages of mapreduce
mapper1 = Mapper(user_item_counts, oversampling=10)
mapreduce(datafile, mapout1, mapper=mapper1, reducer=reducer)
mapreduce(datafile, mapout2, mapper=indicator_mapper)  # map the data again
mapreduce([mapout1, mapout2], outfile, reducer=indicator_reducer)

In [33]:
class ExternalSchedule(object):

    def __init__(self, filepath):
        self.filepath = filepath

    def generate_samples(self, data):
        f = open(self.filepath)
        samples = [map(str, line.strip().split()) for line in f]
        random.shuffle(samples)  # important!
        for u, i, j in samples:
            yield u, i, j

In [34]:
sampler = ExternalSchedule(outfile)  # schedule is one-indexed

# Complete Latent Factor Model with Binary Classification

A simple non-biased latent factor model that is wrapped into a binary function (sigmoid function) as a base line model, using popularity as the item's sole feature

In [35]:
itemBiases = defaultdict(float)

For each user and item we now have a low dimensional descriptor (representing that user's preferences, and that item's properties), of dimension K.

In [36]:
userGamma = {}
itemGamma = {}

In [37]:
K = 2

In [38]:
for u in train_itemsPerUser:
    userGamma[u] = [random.random() * 0.1 - 0.05 for k in range(K)]

In [39]:
for i in train_usersPerItem:
    itemGamma[i] = [random.random() * 0.1 - 0.05 for k in range(K)]

We'll use another library in this example to perform gradient descent. This library requires that we pass it a "flat" parameter vector (theta) containing all of our parameters. This utility function just converts between a flat feature vector, and our model parameters, i.e., it "unpacks" theta into our offset and bias parameters.

In [40]:
def unpack(theta):
    global itemBiases
    global userGamma
    global itemGamma
    index = 0
    itemBiases = dict(zip(train_items, theta[0:index + train_nItems]))
    index += train_nItems
    for u in train_users:
        userGamma[u] = theta[index:index + K]
        index += K
    for i in train_items:
        itemGamma[i] = theta[index:index + K]
        index += K

In [41]:
def inner(x, y):
    return sum([a*b for a,b in zip(x,y)])

## Sigmoid Function

\begin{equation*}
\sigma(x) = \frac{1}{1 + e^{-x}}
\end{equation*}

In [42]:
def sigmoid(x):
    #Numerically stable sigmoid function.
    #Taken from: https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    if x >= 0:
        z = np.exp(-x)
        return 1 / (1 + z)
    else:
        # if x is less than zero then z will be small, denom can't be
        # zero because it's 1+z.
        z = np.exp(x)
        return z / (1 + z)

## Prediction Function

\begin{equation*}
f(u, i, j) = \gamma_u \gamma_i + \beta_i - (\gamma_u \gamma_j + \beta_j)
\end{equation*}

\begin{equation*}
p(i >_u j) = \sigma(f(u, i, j))
\end{equation*}

In [43]:
def prediction(user, item_i, item_j):
    return inner(userGamma[user], itemGamma[item_i]) + itemBiases[item_i] - (inner(userGamma[user], itemGamma[item_j]) + itemBiases[item_j]) 

## Cost Function

\begin{equation*}
\text{BPR-OPT} := \text{argmax} \ln(\sigma(\gamma_u \gamma_i + \beta_i - (\gamma_u \gamma_j + \beta_j)))
\end{equation*}

\begin{equation*}
\text{Cost Function}:= \sum_{u,i,j} -ln\left( 1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)} \right)
\end{equation*}

In [44]:
def trim(u, i, j):
    return u[1:len(u)-1], i[2:len(i)-2], j[1:len(j)-2]

In [45]:
def cost(theta):
    unpack(theta)
    cost = 0
    for u, i, j in sampler.generate_samples(data):
        u, i, j = trim(u, i ,j)
        x = prediction(u, i, j)

        cost += np.log(sigmoid(x))
        
    print(-cost)
        
    return -cost

## Gradient Ascent

\begin{equation*}
\frac{\partial f}{\partial \gamma_{u,k}} = \frac{(\gamma_{i,k} - \gamma_{j,k}) \cdot e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial \gamma_{i,k}} = \frac{\gamma_{u,k} \cdot e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial \gamma_{j,k}} = -\frac{\gamma_{u,k} \cdot e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial \beta_i} = \frac{e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial \beta_j} = -\frac{e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

In [46]:
def derivative(theta):
    unpack(theta)
    dItemBiases = defaultdict(float)
    dUserGamma = {}
    dItemGamma = {}
    for u in train_users:
        dUserGamma[u] = [0.0 for k in range(K)]
    for i in train_items:
        dItemGamma[i] = [0.0 for k in range(K)]
    for u, i, j in sampler.generate_samples(data):
        u, i, j = trim(u, i ,j)
        x = prediction(u, i ,j)
        dbase = np.exp(-x) / (1 + np.exp(-x))
        dItemBiases[i] += dbase
        dItemBiases[j] += -dbase
        for k in range(K):
            dUserGamma[u][k] += (itemGamma[i][k] - itemGamma[j][k]) * dbase
            dItemGamma_k = userGamma[u][k] * dbase
            dItemGamma[i][k] += dItemGamma_k
            dItemGamma[j][k] += -dItemGamma_k
    dtheta = [dItemBiases[i] for i in train_items]
    for u in train_users:
        dtheta += dUserGamma[u]
    for i in train_items:
        dtheta += dItemGamma[i]
    return np.array(dtheta)

In [ ]:
sy.optimize.fmin_l_bfgs_b(cost, [0.0]*train_nItems + # Initialize beta
                                [random.random() * 0.1 - 0.05 for k in range(K*(train_nUsers + train_nItems))], # Gamma
                             derivative)

19961.3778207
-0.000643211209731
-0.000477306531738
-0.00119681007503
-0.00154904359249
0.000481803333729
0.00592869285237
-0.00348215101454
0.00176195190981
-0.00032895825836
-0.000540997928424
-0.00317054017454
-0.000783528112501
0.000286952211318
-0.000870038399055
-0.00279138887789
-0.00309072247955
-0.00119872000645
0.00225574725162
0.00121702548059
-0.000345559972359
0.000263351108728
-0.00305915890749
0.000314765229462
0.00135794737113
-0.000594357584738
7.36583575952e-05
0.00191114952634
0.000556886669617
0.000646627097784
0.00199339403101
0.000577627769546
-0.00147803653598
-0.00290792626381
-0.000611784765661
0.000920631149416
0.00190887920537
-0.000327480762391
-0.00031625920589
-0.00255857166932
0.000516684948855
0.0024502180927
-0.000955489669103
0.000687996623163
-0.000946508046963
-6.52178746694e-06
-0.000282266416446
-0.00251780684612
-0.00173252119615
0.000661001629752
-0.00200798007457
0.000121100762381
-0.00060134119824
-0.00492052402205
0.00081486646761
0.0007022298

-0.00143560553211
-0.000518863315209
-0.00234616818362
0.00130264373545
-0.000895072802919
0.00182348928864
0.000918213195692
0.00158796839633
0.00205970298894
-0.000297616353405
0.00112394637608
0.000616218934513
0.0012593348764
-0.000571038883924
-0.00126582556609
-0.000306154249377
-0.000298924535765
0.00057525146378
-6.69527710266e-05
0.00201734883615
0.000699033224652
0.00227325220038
-0.000688058188371
-0.00202481643173
0.000341177122733
0.00186690084689
0.00147840439191
0.00424998180812
-5.62227210271e-05
-0.000196920354383
0.000189755235727
-0.000820709331336
-0.00208868179116
0.00131965664078
0.00432326335062
0.000530814002761
0.00020393672047
-0.000693107097073
-0.000745782560491
0.000257264504434
-0.00129060531719
0.000265581062798
-0.000315556526124
0.00151949437409
0.000287481283991
-6.80903378794e-06
0.0022672714068
0.00256692052837
-0.00190594998302
-0.00193759828583
-7.1265672825e-05
-0.0020319507853
-0.00255555651028
0.00339261574184
-0.000363403643038
0.00273223326384

-1.02460686869e-05
0.00328441783495
-0.00120310245273
-0.000763642110047
0.000357689923022
0.00332960408915
-0.00203284122835
-0.00437931117757
0.00215794369834
-0.00182709588496
0.000723836360875
-0.00100811922592
6.03311558534e-05
-0.00508527093973
-0.000282081871868
5.40598589203e-05
7.68320878475e-05
0.000635880449158
-0.000803401282741
-0.000591534385196
0.000446185698157
-0.000642391267803
-0.000247081283122
0.00248616711636
4.40855671066e-05
0.00384973235873
0.000937742326531
-0.00179018005228
-0.00355780857534
0.000937480959181
-0.00353072752228
-0.00230078117224
0.000610127139763
0.00155979392739
0.000692252760038
-0.000323653296948
-0.000828143273969
-0.00136179876098
-0.000769580273589
0.00162658280069
0.00114139581597
0.00158689430495
7.30838262861e-05
0.000122863295514
0.0002676851506
-0.00422199206582
-0.00199459640924
-0.000156359112597
-0.00154650353798
-0.00147476513821
-0.000463808576844
-0.00163457220192
-0.000109642611414
-2.97826354922e-05
0.00173317121667
0.001248

0.000139338771202
-0.000306112554622
-0.000758735829232
-0.000200030785126
0.0013872669303
-8.20794468825e-06
0.00085011749663
0.000854914950083
0.00155101345476
0.00104605911284
-4.0234714349e-05
-0.000354125209648
-0.00258796490197
-0.00087762533385
-0.000346763773415
0.000690556357895
0.000778451832469
-0.000555928540563
0.00126037872979
-0.00107003876974
0.000576112143574
-0.00108810550181
0.000924878755724
0.00383512983644
0.00173013788477
0.000451539245097
-0.000103128405395
0.00128410601875
0.00180964958885
0.000321612853983
0.000986753267767
0.0048542056681
-0.00516762961425
-8.33422894378e-06
0.00403607482199
-0.00288496788594
0.000409629404054
-0.00157595782779
-0.00231307258497
0.000413220786046
-0.00236833808251
0.00110687625466
0.000776170476251
0.000597808801117
-0.000522625338098
-0.00547702928585
0.00185017687022
0.00072303483152
-0.000726662413945
0.00304308391715
-0.00398906792135
0.00129060594021
-0.00128488697054
0.00369355385112
0.00021448372445
-0.0034652214214
0.

4.47076592562e-05
-0.00184403812973
-0.00173192451154
0.000946020751616
0.0027020316245
-0.00234584559604
-0.000169073136597
0.00474622721801
-0.00193464786648
-0.000192072683555
-0.00324558868608
-0.000308601609902
0.00108224480285
-0.00390670457766
-0.00333808796348
-0.00229854188671
0.00167167250367
-0.00042006407732
0.000282610529157
0.00029881329357
-0.00109575595228
-0.000528576385607
-0.000802945108339
0.000333614962528
-0.000894355687545
-5.23739587748e-05
0.000154047938983
0.00336086640076
-0.0013569808554
-0.00038388187276
0.00219491647439
0.00157530067341
0.00193729138117
0.000328394129757
0.00286585381782
8.47604559448e-05
0.000444691656279
0.00153971267932
-0.000149553786752
-0.000164278109244
-0.00144445615016
0.00266110485483
0.000982968352986
0.00340195684304
-0.00353858943357
0.000503199173989
-0.00169520180285
-0.00348412253705
-5.24960443679e-05
0.00212007779674
-0.00228448617466
0.00330967422705
-0.000289881531283
-0.00137646432407
0.000166306620871
0.00150381606519

...

-0.00625126331684
0.00300825690803
0.000223480822108
0.00195082199592
0.00287980751274
-0.000709292345816
0.000874894527858
0.00049537831883
0.00110756886829
-0.00044064868403
-0.000334450008122
0.00320219806774
-0.000323405510511
0.00417407952162
0.00133070201114
-0.000498782381435
0.000603838449205
-0.000784968573296
-0.00317654517344
0.000499486836818
-0.000658036155492
0.000122602507514
0.000967919966703
-0.00215330294193
-0.00105849124323
0.000834525527699
0.00163231393064
-0.00141724210567
0.000468285948293
0.00137252183991
0.000102793645862
0.000185639167333
0.000142598067692
-0.00257180185422
0.00144106157991
-0.00110263685491
-0.00565288353106
0.0019631529578
0.000514264842059
-0.00139549095792
0.000400404822975
-0.000158968954431
-0.0022750063839
-0.000806382468651
0.000836755062894
-0.00157103489477
0.00259509686713
0.00264508020203
0.00160042858364
-0.000890184007219
0.000424705674029
-0.00388032417358
1.67018214849e-05
-0.000216417309919
0.000296618990231
-0.00073239222500

0.00139715936185
-0.00241666046258
0.0016406548521
0.00114107408234
0.00213628621953
0.000427164879439
1.65503801206e-05
-0.00134181531867
-0.00466484670164
-0.00439116152204
7.05229946698e-05
0.0028980547262
0.000254903350342
-0.0018586698728
-0.00165088833313
-0.00013330111604
0.000642528352594
-0.0028586468631
-0.000341777275247
-0.000229326945362
0.0022361114571
0.00359298410299
0.000812263118973
0.0010838648922
7.1624242725e-05
0.00076670928957
2.04390488516e-05
0.00378211618879
0.000715924166682
0.000402569984222
0.000161729841078
-0.00311110091057
-0.000265369443571
-0.00030207991568
-0.000514584285572
0.00145700673687
-0.00149981028983
-0.00169391637433
0.000365527893453
0.00134274007609
-0.000183029974542
0.000408041871034
0.00155009567094
0.000476193231696
0.00046564194646
0.00303092104323
-0.000321432030307
-0.000806276043114
0.0016657628628
-0.000703324127384
0.000473922439612
0.00170075906751
0.00172686427987
-0.000982036826859
0.000221802042277
0.000383738282362
-5.288304

-0.00207807525898
0.00215699568663
-0.00259271813913
0.000898184476655
-0.00703750508458
0.00195121497186
0.000687597505688
-0.000609891231389
-0.000157619070116
0.000550480143764
0.000714668499459
-0.000845136596736
-0.000461149948236
-0.00094996072705
0.00142232716839
0.00147099683726
-0.00103090087727
-5.92520855054e-05
0.00268677124335
0.000112517433942
0.00197163113057
0.00359101528566
-0.00381363970243
-0.000204826427153
-0.000812041423086
-0.00196760841486
0.00259706732106
-0.00309926623564
-0.00079752365243
-0.000114341061413
-0.001732362521
-0.000737537517557
0.000917853733162
0.00141729145715
-0.00060115097179
-0.00118737801463
0.00131193457409
0.00165304780967
0.000313403308873
0.000604291782363
-0.00108393865539
-0.00184648744946
0.0015907801071
-0.00162785084407
-0.00106792454069
-0.000210617147031
0.00455513621953
-0.00130968526191
-9.66928958227e-05
-0.00038331539253
0.000503707524603
-0.000857453200927
-2.70206484727e-05
-0.00258773212049
-0.000382691352458
0.0002976073

0.00134224402774
0.00126416779289
-8.2881050034e-05
-0.000715979705462
0.000676417705416
-0.000491879339999
0.000485918906615
0.00294106126218
-0.000358405618752
-0.000213436472291
0.000263996129891
-0.00148980784062
0.00199975109147
-0.00117219262268
0.00319622725612
-0.00162749358326
-0.000351870822209
0.000380866339
0.000140071228556
0.00258355135192
-0.000299527297079
-0.00169617298386
-0.00346368039149
0.000729399619005
-0.00150871256254
-0.00113687157555
0.001459952726
-0.000318247383963
0.000348221040958
0.00110873351074
0.000228081622514
-0.000349000253163
0.00072205320535
-0.000452254138427
-0.000694846565527
0.00323454837812
-0.000920063285327
0.00253717107935
-0.00132611321481
0.00223439492641
-0.000105838801527
0.000729270975016
-0.00120174812969
2.9687240965e-05
0.000179114084217
-0.000900925435982
-0.0014688433925
-0.000736862817856
0.000526461419028
-0.00224709346943
-0.000140990336594
0.00223794714139
0.00112414933558
-0.00133426502043
-0.00275228330578
-0.0005399036727

0.00156852593093
-0.000325984052792
-0.000158778376876
-0.00274756094479
-0.0020358556197
-0.000803422980337
0.000116927980066
0.00128987404661
-0.000595525939221
-6.42930858415e-05
-0.00159058216217
-0.000123658492802
-0.00189456854574
0.00356407749403
-0.00187087576173
0.000989690081925
0.00208999915469
-0.00137247324725
0.00139314497898
-0.000873475863373
0.000481502138997
-6.37841058362e-05
0.0027675901198
0.00136301915287
0.00053335012607
-0.00289771320308
7.61676469682e-05
0.00351891043778
-0.00235993885283
0.00258089979114
-0.000386243379364
0.00336434570819
0.000846724096835
0.000711011926891
1.58391913346e-05
0.000992846035792
0.00251267152868
0.00245549832215
0.00103819850002
0.00255859600535
-0.000364119979136
-0.00188294873855
0.000334444255121
0.000581848248481
0.00222314846741
0.00128411517315
-0.00155900314543
0.000242456733193
0.00484908829894
-0.000615481320799
-0.00114120318605
0.000549473155519
0.00215396231093
-0.000776148406618
-0.000615684759422
0.000358185348889
